<a href="https://colab.research.google.com/github/NLU-Project/emnlp2020-debiasing-unknown/blob/main/SBIC_Civil_Covert_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Dependecies
!pip install datasets
!pip install tensorflow
!pip install pandas-profiling
from datasets import load_dataset
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np

[Social bias frames](https://huggingface.co/datasets/social_bias_frames]) This is the challenge dataset that features microaggressions. "Social Bias Inference Corpus (SBIC) which contains about 150k structured annotations of social media posts, covering over 34k implications about a thousand demographic groups." created by [Washington CS program researchers](https://homes.cs.washington.edu/~msap/social-bias-frames/#:~:text=What%20are%20Social%20Bias%20Frames,standards%20to%20hire%20more%20women.%22). Its mostly reddit, but has posts from Reddit, Gab, and Stormfront posts. 

Annotators: The annotators are Amazon Mechanical Turk workers aged 36±10 years old. The annotators consisted of 55% women, 42% men, and <1% non-binary and 82% identified as White, 4% Asian, 4% Hispanic, 4% Black.

For our model evaluations we will use 



*   post: a string containing the text of the post that was annotated
*   offensiveYN: a string indicating if the post could be offensive to anyone. This is a categorical variable with three possible answers, ‘1.0’ if yes, ‘0.5’ if maybe, ‘0.0’ if no.

Data Splits
To ensure that no post appeared in multiple splits, the curators defined a training instance as the post and its three sets of annotations. They then split the dataset into train, validation, and test sets (75%/12.5%/12.5%).

| name | train | validation |	test | combined
| --- | --- | --- | --- | --- |
| default	| 112900 | 16738	| 17501 | 147139 | 



In [11]:
sbic_train = load_dataset('social_bias_frames', split='train')
sbic_val = load_dataset('social_bias_frames', split='validation')

Using custom data configuration default
Reusing dataset social_bias_frames (/root/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510)
Using custom data configuration default
Reusing dataset social_bias_frames (/root/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510)


In [12]:
!pip3 install sweetviz

In [13]:
import sweetviz as sv
df_sbic_train = sbic_train.to_pandas()
df_sbic_val = sbic_val.to_pandas()

In [18]:
my_report = sv.compare([df_sbic_train, "Training Data"], [df_sbic_val, "Validation Data"])
my_report.show_html("report_sbic_train_vs_val.html")

                                             |          | [  0%]   00:00 -> (? left)

Report /report_sbic_train_vs_val.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
df_sbic_train


,whoTarget,intentYN,sexYN,sexReason,offensiveYN,annotatorGender,annotatorMinority,sexPhrase,speakerMinorityYN,WorkerId,HITId,annotatorPolitics,annotatorRace,annotatorAge,post,targetMinority,targetCategory,targetStereotype,dataSource
0,0.0,0.66,0.0,,1.0,woman,,,,-8935932304856669427,363A7XIFV4G2799C5V96YERJA9AVAM,liberal,white,45.0,RT @_LexC__: I'm convinced that some of y'all ...,,,,t/davidson
1,0.0,0.66,0.0,,0.5,man,,,,6347880360297734464,363A7XIFV4G2799C5V96YERJA9AVAM,mod-liberal,white,35.0,RT @_LexC__: I'm convinced that some of y'all ...,,,,t/davidson
2,0.0,0.33,0.0,,0.5,man,,,,-7452610791699819066,363A7XIFV4G2799C5V96YERJA9AVAM,liberal,asian,23.0,RT @_LexC__: I'm convinced that some of y'all ...,,,,t/davidson
3,1.0,1.0,0.0,,1.0,man,,,0.0,-500114755446676507,3JTPR5MTZS6RLS3JBV4IOU0G2X35K5,liberal,white,25.0,RT @iBeZo: Stupid fucking nigger LeBron. You f...,black folks,race,all stupid,t/davidson
4,1.0,1.0,0.0,,1.0,man,,,0.0,-500114755446676507,3JTPR5MTZS6RLS3JBV4IOU0G2X35K5,liberal,white,25.0,RT @iBeZo: Stupid fucking nigger LeBron. You f...,black folks,race,are not people but apes.,t/davidson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112895,0.0,0.33,0.0,,1.0,na,,,,1558992385113521549,3G57RS03HHZW4CZTMPYEOYEMNVI25F,na,na,35.0,Thought you fellas might like to explain to so...,,,,r/MensRights
112896,,0.0,0.0,,0.0,man,,,,-5870881806826799680,3G57RS03HHZW4CZTMPYEOYEMNVI25F,liberal,white,35.0,Thought you fellas might like to explain to so...,,,,r/MensRights
112897,,0.0,0.0,,0.0,man,pacific islander,,,-768683606829583421,38O9DZ0A62HSNGZ7R88JG4SMJQT26V,liberal,other,36.0,"If female voters were discounted, Labour would...",,,,r/MensRights
112898,,0.0,0.0,,0.0,man,,,,-5870881806826799680,38O9DZ0A62HSNGZ7R88JG4SMJQT26V,liberal,white,35.0,"If female voters were discounted, Labour would...",,,,r/MensRights


In [ ]:
ds = tfds.load('CivilComments')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: ignored

In [ ]:
ds

{'test': <PrefetchDataset element_spec={'identity_attack': TensorSpec(shape=(), dtype=tf.float32, name=None), 'insult': TensorSpec(shape=(), dtype=tf.float32, name=None), 'obscene': TensorSpec(shape=(), dtype=tf.float32, name=None), 'severe_toxicity': TensorSpec(shape=(), dtype=tf.float32, name=None), 'sexual_explicit': TensorSpec(shape=(), dtype=tf.float32, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None), 'threat': TensorSpec(shape=(), dtype=tf.float32, name=None), 'toxicity': TensorSpec(shape=(), dtype=tf.float32, name=None)}>,
 'train': <PrefetchDataset element_spec={'identity_attack': TensorSpec(shape=(), dtype=tf.float32, name=None), 'insult': TensorSpec(shape=(), dtype=tf.float32, name=None), 'obscene': TensorSpec(shape=(), dtype=tf.float32, name=None), 'severe_toxicity': TensorSpec(shape=(), dtype=tf.float32, name=None), 'sexual_explicit': TensorSpec(shape=(), dtype=tf.float32, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None), 'threat': T

In [ ]:
tfds.as_dataframe(ds['train'].take(5))

,identity_attack,insult,obscene,severe_toxicity,sexual_explicit,text,threat,toxicity
0,0.000000,0.000000,0.000000,0.000000,0.000000,"b""btw, Globe, your new comment section is lame. \n\nI hit yes everytime because I believe EVERYONE has the right to say whatever they want! I'd rather hear the bad stuff than bury my head in the sand like our snowflakes.""",0.000000,0.200000
1,0.000000,0.000000,0.000000,0.000000,0.000000,"b""If at first you don't succeed...try again: http://www.seattletimes.com/business/boeing-aerospace/will-787-program-ever-show-an-overall-profit-analysts-grow-more-skeptical/""",0.000000,0.000000
2,0.000000,0.700000,0.000000,0.000000,0.000000,"b""You don't understand what leaking is. By the time Mr. Comey gave his unclassified information to his friend, he was a private citizen - thanks to Mr. Trump. There is no law or regulation about a private citizen discussing unclassified information with anyone.\n\nTrump is a psychotic and a liar. He can't be removed from office soon enough.\n\nTry again?""",0.000000,0.700000
3,0.000000,0.000000,0.000000,0.000000,0.000000,b'duties come before rights.\nthey have a duty to earn it if they can work.',0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,"b'Based upon the activities of the OHA I have always been of the opinion that being ""cray cray"" was a prerequisite for candidacy. Apparently local voters prize the attribute.'",0.000000,0.000000


In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
builder = tfds.text.CivilComments(config='CivilCommentsCovert')

In [ ]:
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/48074 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/civil_comments/CivilCommentsCovert/1.2.2.incompleteKIEY3X/civil_comments-train…

Generating test examples...:   0%|          | 0/2455 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/civil_comments/CivilCommentsCovert/1.2.2.incompleteKIEY3X/civil_comments-test.…

Dataset civil_comments downloaded and prepared to ~/tensorflow_datasets/civil_comments/CivilCommentsCovert/1.2.2. Subsequent calls will reuse this data.
<PrefetchDataset element_spec={'article_id': TensorSpec(shape=(), dtype=tf.int32, name=None), 'asian': TensorSpec(shape=(), dtype=tf.float32, name=None), 'atheist': TensorSpec(shape=(), dtype=tf.float32, name=None), 'bisexual': TensorSpec(shape=(), dtype=tf.float32, name=None), 'black': TensorSpec(shape=(), dtype=tf.float32, name=None), 'buddhist': TensorSpec(shape=(), dtype=tf.float32, name=None), 'christian': TensorSpec(shape=(), dtype=tf.float32, name=None), 'covert_emoticons_emojis': TensorSpec(shape=(), dtype=tf.float32, name=None), 'covert_humor': TensorSpec(shape=(), dtype=tf.float32, name=None), 'covert_masked_harm': TensorSpec(shape=(), dtype=tf.float32, name=None), 'covert_microaggression': TensorSpec(shape=(), dtype=tf.float32, name=None), 'covert_obfuscation': TensorSpec(shape=(), dtype=tf.float32, name=None), 'covert_polit

In [ ]:
len(ds)

48074